In [1]:
# import necessary libraries
import pandas as pd, numpy as np, psycopg2;

###### VIEW DATAFRAME

In [2]:
raw_df = pd.read_excel('Online Retail.xlsx') # link - 'https://archive.ics.uci.edu/dataset/352/online+retail'
raw_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


###### RENAME DATAFRAME COLUMNS FOLLOWING RECOMMENDED NAMEING CONVENTIONS

In [3]:
new_columns = ['invoice_no', 'stockcode', 'description', 'quantity', 'invoice_date', 'unitprice', 'customer_id', 'country']
raw_df.columns = new_columns
raw_df.head()

,invoice_no,stockcode,description,quantity,invoice_date,unitprice,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
print(raw_df.shape)
raw_df.info()

(541909, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice_no    541909 non-null  object        
 1   stockcode     541909 non-null  object        
 2   description   540455 non-null  object        
 3   quantity      541909 non-null  int64         
 4   invoice_date  541909 non-null  datetime64[ns]
 5   unitprice     541909 non-null  float64       
 6   customer_id   406829 non-null  float64       
 7   country       541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


###### CHECK FOR MISSING VALUES

In [5]:
raw_df.isna().sum()

invoice_no           0
stockcode            0
description       1454
quantity             0
invoice_date         0
unitprice            0
customer_id     135080
country              0
dtype: int64

#### DATA CLEANING

<small>_check for different value types in individual columns/attributes._<small>

In [6]:
for col in raw_df.columns:
    type_counts = raw_df[col].apply(type).value_counts()
    print(f'{col}: {type_counts}')

invoice_no: invoice_no
<class 'int'>    532618
<class 'str'>      9291
Name: count, dtype: int64
stockcode: stockcode
<class 'int'>    487036
<class 'str'>     54873
Name: count, dtype: int64
description: description
<class 'str'>      540454
<class 'float'>      1454
<class 'int'>           1
Name: count, dtype: int64
quantity: quantity
<class 'int'>    541909
Name: count, dtype: int64
invoice_date: invoice_date
<class 'pandas._libs.tslibs.timestamps.Timestamp'>    541909
Name: count, dtype: int64
unitprice: unitprice
<class 'float'>    541909
Name: count, dtype: int64
customer_id: customer_id
<class 'float'>    541909
Name: count, dtype: int64
country: country
<class 'str'>    541909
Name: count, dtype: int64


_<small>we can see that stockcode, descrtption and invoice_no contail multiple data types in their columns.
**Description**:
from earlier, we saw that the description attribute contains null values; pandas recognize missing data as float so we need to confirm if there are really numbers with decimal points or null values read as float.
But first we cast the entire column as string.<small>_

In [8]:
disc_type_errors = raw_df[raw_df['description'].apply(type).isin([float,int]) & (raw_df['description'].isna() == False) ]

# convert int field to string
raw_df.loc[disc_type_errors.index, 'description'] = raw_df.loc[disc_type_errors.index, 'description'].astype(str)

In [9]:
print(f'''
    isna sum: {raw_df['description'].isna().sum()},
    float sum: {(raw_df['description'].apply(type) == float).sum()}
    '''
)

# null values are read as float


    isna sum: 1454,
    float sum: 1454
    


In [10]:
# clean stock code column
stockcode_int = raw_df[raw_df['stockcode'].apply(type) == int]
stockcode_str = raw_df[raw_df['stockcode'].apply(type) == str]
print(stockcode_int['stockcode'].head(), '\n' , stockcode_str['stockcode'].head())

1    71053
5    22752
6    21730
7    22633
8    22632
Name: stockcode, dtype: object 
 0     85123A
2     84406B
3     84029G
4     84029E
45      POST
Name: stockcode, dtype: object


In [11]:
raw_df.loc[stockcode_int.index, 'stockcode'] = raw_df.loc[stockcode_int.index,'stockcode'].astype(str)
raw_df[raw_df['stockcode'].apply(type) == int].shape

(0, 8)

In [12]:
# clean InvoiceNo column
InvoiceNo_int = raw_df[raw_df['invoice_no'].apply(type) == int]
InvoiceNo_str = raw_df[raw_df['invoice_no'].apply(type) == str]
print(InvoiceNo_int['invoice_no'].head(), '\n' , InvoiceNo_str['invoice_no'].head())

0    536365
1    536365
2    536365
3    536365
4    536365
Name: invoice_no, dtype: object 
 141    C536379
154    C536383
235    C536391
236    C536391
237    C536391
Name: invoice_no, dtype: object


In [13]:
raw_df.loc[InvoiceNo_int.index,'invoice_no'] = raw_df.loc[InvoiceNo_int.index,'invoice_no'].astype(str)
raw_df[raw_df['invoice_no'].apply(type) == int].shape

(0, 8)